# Types

On fait pointer les built in attribute vers le premier mot type  

ACHTUNG pour conversation et date, les attributes ont des backspace autour du nom quand la doc est generée automatiquement, je les ai retirés à la main !

### Old, to delete once the new cell is functionnal

In [1]:
# import re
# from neo4j import GraphDatabase
# 
# # Neo4j connection
# uri = "bolt://localhost:7687"
# username = "neo4j"
# password = "password"
# driver = GraphDatabase.driver(uri, auth=(username, password))
# 
# def normalize_type_name(type_str):
#     return re.split(r"[\s,()\[\]{}]+", type_str.strip())[0].lower()
# 
# 
# # === Markdown Parsing ===
# def parse_datatypes_file(path):
#     with open(path, "r", encoding="utf-8") as f:
#         content = f.read()
# 
#     sections = re.split(r"\n###\s+(\w+)\n", content)[1:]
#     types = []
# 
#     for i in range(0, len(sections), 2):
#         type_name = sections[i].strip()
#         block = sections[i + 1]
# 
#         data = {
#             "name": type_name,
#             "definition": None,
#             "comments": None,
#             "litteral_declaration": None,
#             "other_declaration": None,
#             "example": None,
#             "attributes": []
#         }
# 
#         # Standard fields
#         for key in data.keys():
#             if key in {"name", "attributes"}:
#                 continue
#             regex = rf"\* \*\*{key.replace('_', ' ').capitalize()}:?\*\* (.+?)(?=\n\* \*\*|\n```|\Z)"
#             match = re.search(regex, block, re.S)
#             if match:
#                 data[key] = match.group(1).strip()
# 
#         # Built-in attributes
#         attr_block_match = re.search(
#             r"(?m)^ *\* \*\*Built-in attributes:\*\*\s*(?:\n|)(.*?)(?=\n *\* \*\*|\n *\[Top of the page\]|\n```|\n\n|\Z)",
#             block,
#             re.S
#         )
# 
# 
#         if attr_block_match:
#             attr_lines = attr_block_match.group(1).strip().splitlines()
#             for line in attr_lines:
#                 attr_match = re.match(r"\*?\s*(\w+)\s*\(type\s*=\s*([^)]+)\):\s*(.+)", line.strip())
#                 if attr_match:
#                     attr_name = attr_match.group(1).strip()
#                     attr_type = attr_match.group(2).strip()
#                     attr_desc = attr_match.group(3).strip()
#                     data["attributes"].append({
#                         "name": attr_name,
#                         "type": attr_type,
#                         "description": attr_desc,
#                         "full_line": line.strip()
#                     })
# 
#         # Debug: no attributes found
#         if not data["attributes"]:
#             print(f"⚠️ No attributes found for type: {type_name}\n--- Block Start ---\n{block[:300]}...\n--- Block End ---\n")
# 
#         # Example code
#         example_match = re.search(r"```(.*?)```", block, re.S)
#         if example_match:
#             data["example"] = example_match.group(1).strip()
# 
#         types.append(data)
# 
#     return types
# 
# 
# 
# # === Phase 1: Create Type nodes and BuiltInAttribute nodes ===
# def insert_types_and_attributes(types_data):
#     with driver.session() as session:
#         for dtype in types_data:
#             session.run("""
#                 MERGE (t:Type {name: $name})
#                 SET t.description = $definition,
#                     t.comments = $comments,
#                     t.litteral_declaration = $litteral_declaration,
#                     t.other_declaration = $other_declaration,
#                     t.example = $example
#             """, **dtype)
# 
#             for attr in dtype["attributes"]:
#                 session.run("""
#                     MATCH (t:Type {name: $type_name})
#                     CREATE (a:BuiltInAttribute {
#                         name: $attr_name,
#                         description: $attr_desc,
#                         parent_type: $type_name,
#                         target_type: $attr_type,
#                         full_line: $attr_full_line
#                     })
#                     MERGE (t)-[:HAS_ATTRIBUTE]->(a)
#                 """, attr_name=attr["name"],
#                      attr_desc=attr["description"],
#                      attr_type=normalize_type_name(attr["type"]), #on recupere seulement le premier mot
#                      type_name=dtype["name"],
#                      attr_full_line=attr["full_line"]),
# 
# 
# 
# # === Phase 2: Create OF_TYPE relationships after all Type nodes exist ===
# def create_of_type_relations():
#     with driver.session() as session:
#         # Get all BuiltInAttributes with a target_type
#         result = session.run("""
#             MATCH (a:BuiltInAttribute)
#             WHERE a.target_type IS NOT NULL
#             RETURN a.name AS attr_name, a.description AS attr_desc, a.target_type AS target_type
#         """)
# 
#         for record in result:
#             attr_name = record["attr_name"]
#             attr_desc = record["attr_desc"]
#             target_type = record["target_type"]
# 
#             # Try to find matching Type node
#             check = session.run("""
#                 MATCH (t:Type {name: $target_type})
#                 RETURN count(t) AS count
#             """, target_type=target_type).single()["count"]
# 
#             if check == 0:
#                 print(f"⚠️ Missing Type: No Type node found for target_type='{target_type}' (attribute='{attr_name}')")
#             else:
#                 # Create the OF_TYPE relationship
#                 session.run("""
#                     MATCH (a:BuiltInAttribute {name: $attr_name, description: $attr_desc})
#                     MATCH (t:Type {name: $target_type})
#                     MERGE (a)-[r:OF_TYPE]->(t)
#                     SET r.description = a.full_line
#                 """, attr_name=attr_name, attr_desc=attr_desc, target_type=target_type)
# 
# 
# 
# # === Optional Cleanup (remove temporary fields) ===
# def cleanup_temp_fields():
#     with driver.session() as session:
#         session.run("""
#             MATCH (a:BuiltInAttribute)
#             REMOVE a.target_type, a.parent_type, a.full_line
#         """)
# 
# 
# # === Run the pipeline ===
# types_data = parse_datatypes_file("DataTypes.md")
# insert_types_and_attributes(types_data)
# create_of_type_relations()
# cleanup_temp_fields()
# driver.close()


### New Type generation

In [2]:
import re
from neo4j import GraphDatabase

# Neo4j connection
uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"
driver = GraphDatabase.driver(uri, auth=(username, password))

def normalize_type_name(type_str):
    return re.split(r"[\s,()\[\]{}]+", type_str.strip())[0].lower()




# === Markdown Parsing ===
def parse_datatypes_file(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()

    sections = re.split(r"\n###\s+(\w+)\n", content)[1:]
    types = []

    for i in range(0, len(sections), 2):
        type_name = sections[i].strip()
        block = sections[i + 1]

        data = {
            "name": type_name,
            "definition": None,
            "comments": None,
            "litteral_declaration": None,
            "other_declarations": None,
            "example": None,
            "attributes": []
        }

        # Standard fields
        for key in data.keys():
            if key in {"name", "attributes"}:
                continue
            regex = rf"\* \*\*{key.replace('_', ' ').capitalize()}:?\*\*(.*?)(?=\n\s*\* \*\*|\n\[Top of the page\]|\n###|\Z)"
            match = re.search(regex, block, re.S)
            if match:
                data[key] = match.group(1).strip()

        # Built-in attributes
        attr_block_match = re.search(
            r"(?m)^ *\* \*\*Built-in attributes:\*\*\s*(?:\n|)(.*?)(?=\n *\* \*\*|\n *\[Top of the page\]|\n```|\n\n|\Z)",
            block,
            re.S
        )


        if attr_block_match:
            attr_lines = attr_block_match.group(1).strip().splitlines()
            for line in attr_lines:
                attr_match = re.match(r"\*?\s*(\w+)\s*\(type\s*=\s*([^)]+)\):\s*(.+)", line.strip())
                if attr_match:
                    attr_name = attr_match.group(1).strip()
                    attr_type = attr_match.group(2).strip()
                    attr_desc = attr_match.group(3).strip()
                    data["attributes"].append({
                        "name": attr_name,
                        "type": attr_type,
                        "description": attr_desc,
                        "full_line": line.strip()
                    })

        # Debug: no attributes found
        if not data["attributes"]:
            print(f"⚠️ No attributes found for type: {type_name}\n--- Block Start ---\n{block[:300]}...\n--- Block End ---\n")

        # Example code
        # Extract example code block if present
        decl_match = re.search(
            r"\* \*\*Declaration:?\*\*(.*?)(?=\n\s*\* \*\*|\n\[Top of the page\]|\n###|\Z)",
            block,
            re.S
        )

        if decl_match:
            # ✅ Case 1: Declaration exists → process with code blocks
            decl_raw = decl_match.group(1).strip()
            example_lines = []
            inside_code = False

            for line in decl_raw.splitlines():
                line = line.rstrip()
                if line.strip().startswith("```"):
                    inside_code = not inside_code
                    example_lines.append(line)
                    continue

                if inside_code:
                    example_lines.append(line)
                elif line.strip() == "":
                    example_lines.append("")
                else:
                    example_lines.append(f"// {line}")

            data["example"] = "\n".join(example_lines).strip()

        else:
            # ✅ Case 2: No Declaration → fallback to litteral + other, preserving ``` blocks
            example_lines = []
            inside_code = False

            for field in ["litteral_declaration", "other_declaration"]:
                value = data.get(field)
                if value:
                    for line in value.strip().splitlines():
                        line = line.rstrip()
                        if line.strip().startswith("```"):
                            inside_code = not inside_code
                            example_lines.append(line)
                        elif inside_code:
                            example_lines.append(line)
                        elif line.strip() == "":
                            example_lines.append("")
                        else:
                            example_lines.append(f"// {line}")

            data["example"] = "\n".join(example_lines).strip()




        types.append(data)

    return types



# === Phase 1: Create Type nodes and BuiltInAttribute nodes ===
def insert_types_and_attributes(types_data):
    with driver.session() as session:
        for dtype in types_data:
            session.run("""
                MERGE (t:Type {name: $name})
                SET t.description = $definition,
                    t.comments = $comments,
                    t.litteral_declaration = $litteral_declaration,
                    t.other_declarations = $other_declarations,
                    t.example = $example
            """, **dtype)

            for attr in dtype["attributes"]:
                session.run("""
                    MATCH (t:Type {name: $type_name})
                    CREATE (a:BuiltInAttribute {
                        name: $attr_name,
                        description: $attr_desc,
                        parent_type: $type_name,
                        target_type: $attr_type,
                        full_line: $attr_full_line
                    })
                    MERGE (t)-[:HAS_ATTRIBUTE]->(a)
                """, attr_name=attr["name"],
                     attr_desc=attr["description"],
                     attr_type=normalize_type_name(attr["type"]), #on recupere seulement le premier mot
                     type_name=dtype["name"],
                     attr_full_line=attr["full_line"]),



# === Phase 2: Create OF_TYPE relationships after all Type nodes exist ===
def create_of_type_relations():
    with driver.session() as session:
        # Get all BuiltInAttributes with a target_type
        result = session.run("""
            MATCH (a:BuiltInAttribute)
            WHERE a.target_type IS NOT NULL
            RETURN a.name AS attr_name, a.description AS attr_desc, a.target_type AS target_type
        """)

        for record in result:
            attr_name = record["attr_name"]
            attr_desc = record["attr_desc"]
            target_type = record["target_type"]

            # Try to find matching Type node
            check = session.run("""
                MATCH (t:Type {name: $target_type})
                RETURN count(t) AS count
            """, target_type=target_type).single()["count"]

            if check == 0:
                print(f"⚠️ Missing Type: No Type node found for target_type='{target_type}' (attribute='{attr_name}')")
            else:
                # Create the OF_TYPE relationship
                session.run("""
                    MATCH (a:BuiltInAttribute {name: $attr_name, description: $attr_desc})
                    MATCH (t:Type {name: $target_type})
                    MERGE (a)-[r:OF_TYPE]->(t)
                    SET r.description = a.full_line
                """, attr_name=attr_name, attr_desc=attr_desc, target_type=target_type)



# === Optional Cleanup (remove temporary fields) ===
def cleanup_temp_fields():
    with driver.session() as session:
        session.run("""
            MATCH (a:BuiltInAttribute)
            REMOVE a.target_type, a.parent_type, a.full_line
        """)


# === Run the pipeline ===
types_data = parse_datatypes_file("resources/DataTypes.md")
insert_types_and_attributes(types_data)
create_of_type_relations()
cleanup_temp_fields()
driver.close()


⚠️ No attributes found for type: unknown
--- Block Start ---
* **Definition:** TEMPORARY ADDED

```
TEMPORARY ADDED
```
...
--- Block End ---

⚠️ No attributes found for type: emotion
--- Block Start ---
* **Definition:** TEMPORARY ADDED

```
TEMPORARY ADDED
```
...
--- Block End ---

⚠️ No attributes found for type: image
--- Block Start ---
* **Definition:** TEMPORARY ADDED

```
TEMPORARY ADDED
```
...
--- Block End ---

⚠️ No attributes found for type: directory
--- Block Start ---
* **Definition:** TEMPORARY ADDED

```
TEMPORARY ADDED
```
...
--- Block End ---

⚠️ No attributes found for type: font
--- Block Start ---
* **Definition:** TEMPORARY ADDED

```
TEMPORARY ADDED
```
...
--- Block End ---

⚠️ No attributes found for type: predicate
--- Block Start ---
* **Definition:** TEMPORARY ADDED

```
TEMPORARY ADDED
```
...
--- Block End ---

⚠️ No attributes found for type: social_link
--- Block Start ---
* **Definition:** TEMPORARY ADDED

```
TEMPORARY ADDED
```
...
--- Block End -

# all builtin

In [3]:
# import re
# from neo4j import GraphDatabase

# # Neo4j connection configuration
# uri = "bolt://localhost:7687"
# username = "neo4j"
# password = "password"  # Change this to your actual password
# driver = GraphDatabase.driver(uri, auth=(username, password))

# def normalize_type_name(type_str):
#     # First extract content inside backquotes if present
#     match = re.search(r"`([^`]+)`", type_str)
#     if match:
#         type_str = match.group(1)
    
#     # Now normalize the result (remove generics, etc.)
#     return re.split(r"[\s,()<>\[\]{}]+", type_str.strip())[0].lower()


# def parse_species_file(path):
#     with open(path, "r", encoding="utf-8") as file:
#         content = file.read()

#     # Split blocks per species/architecture/skill
#     species_blocks = re.split(r"\n##\s+`?([a-zA-Z0-9_]+)`?\s*\n", content)[1:]
#     species_data = []

#     for i in range(0, len(species_blocks), 2):
#         species_name = species_blocks[i].strip()
#         block = species_blocks[i + 1]

#         # Extract description before "### Variables"
#         desc_match = re.search(r"^\s*(.*?)\s*(?=### Variables|### Actions|$)", block, re.S)
#         description = desc_match.group(1).strip() if desc_match else ""

#         # Extract variables
#         variables = []
#         var_block = re.search(r"### Variables(.*?)(### Actions|----|$)", block, re.S)
#         if var_block:
#             for match in re.finditer(r"\*\*`(.+?)`\*\*\s*\((.+?)\):\s*(.*)", var_block.group(1)):
#                 var_name, var_type, var_desc = match.groups()
#                 variables.append({
#                     "name": var_name.strip(),
#                     "type": normalize_type_name(var_type.strip()),
#                     "description": var_desc.strip()
#                 })

#         # Extract actions
#         actions = []
#         action_block = re.search(r"### Actions(.*?)(----|$)", block, re.S)
#         if action_block:
#             action_parts = re.split(r"#### \*\*`(.+?)`\*\*", action_block.group(1))[1:]
#             for j in range(0, len(action_parts), 2):
#                 action_name = action_parts[j].strip()
#                 details = action_parts[j + 1]

#                 desc_match = re.search(r"\*\*(.*?)\*\*", details, re.S)
#                 return_type_match = re.search(r"\*\*Returned type:\*\* `(.+?)`", details)
#                 action_description = desc_match.group(1).strip() if desc_match else ""
#                 return_type = return_type_match.group(1).strip() if return_type_match else "unknown"

#                 facets = []
#                 facets_block = re.search(r"##### Additional facets:(.*?)(?=\n####|\Z)", details, re.S)
#                 if facets_block:
#                     for facet_match in re.finditer(r"\*\*`(.+?)`\*\* \((.+?)\):\s*(.*?)\s{2,}", facets_block.group(1)):
#                         fname, ftype, fdesc = facet_match.groups()
#                         facets.append({
#                             "name": fname.strip(),
#                             "type": ftype.strip(),
#                             "description": fdesc.strip()
#                         })

#                 actions.append({
#                     "name": action_name,
#                     "description": action_description,
#                     "return_type": return_type,
#                     "facets": facets
#                 })

#         species_data.append({
#             "name": species_name,
#             "description": description,
#             "variables": variables,
#             "actions": actions
#         })

#     return species_data


# def load_to_neo4j(species_data, name_target):
#     with driver.session() as session:
#         for species in species_data:
#             session.run(f"""
#                 MERGE (s:{name_target} {{name: $name}})
#                 SET s.description = $description
#             """, name=species["name"], description=species["description"])

#             for var in species["variables"]:
#                 session.run(f"""
#                     MATCH (s:{name_target} {{name: $species}})
#                     CREATE (v:Variable {{
#                         name: $name,
#                         type: $type,
#                         description: $desc
#                     }})
#                     MERGE (s)-[:HAS_VARIABLE]->(v)
#                 """, species=species["name"], name=var["name"], type=var["type"], desc=var["description"])

#             for act in species["actions"]:
#                 session.run(f"""
#                     MATCH (s:{name_target} {{name: $species}})
#                     CREATE (a:Action {{
#                         name: $name,
#                         return_type: $return_type,
#                         description: $desc
#                     }})
#                     MERGE (s)-[:HAS_ACTION]->(a)
#                 """, species=species["name"], name=act["name"], return_type=act["return_type"], desc=act["description"])

#                 for facet in act.get("facets", []):
#                     session.run("""
#                         MATCH (a:Action {name: $action_name})
#                         CREATE (f:Facet {
#                             name: $name,
#                             type: $type,
#                             description: $desc
#                         })
#                         MERGE (a)-[:HAS_FACET]->(f)
#                     """, action_name=act["name"], name=facet["name"], type=facet["type"], desc=facet["description"])

#         session.run("""
#                 MATCH (v:Variable)
#                 WHERE v.type IS NOT NULL
#                 MATCH (t:Type {name: v.type})
#                 MERGE (v)-[:IS_OF_TYPE]->(t)
#             """)
#         session.run("""
#                 MATCH (a:Action)
#                 WHERE a.return_type IS NOT NULL
#                 MATCH (t:Type {name: a.return_type})
#                 MERGE (a)-[:RETURNS_TYPE]->(t)
#             """)


# if __name__ == "__main__":
#     l = [["built_in_specie", "BuiltInSpecies.md"],["built_in_skill", "BuiltInSkills.md"],["built_in_architecture", "BuiltInArchitectures.md"]]
#     for name_target, file_path in l:
#         species_data = parse_species_file(file_path)
#         load_to_neo4j(species_data, name_target)

#     driver.close()


# New version

In [4]:
import re
from neo4j import GraphDatabase

# Neo4j connection configuration
uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"  # Change this to your actual password
driver = GraphDatabase.driver(uri, auth=(username, password))

def normalize_type_name(type_str):
    # First extract content inside backquotes if present
    match = re.search(r"`([^`]+)`", type_str)
    if match:
        type_str = match.group(1)
    
    # Now normalize the result (remove generics, etc.)
    return re.split(r"[\s,()<>\[\]{}]+", type_str.strip())[0].lower()


def parse_species_file(path):
    with open(path, "r", encoding="utf-8") as file:
        content = file.read()

    # Split blocks per species/architecture/skill
    species_blocks = re.split(r"\n##\s+`?([a-zA-Z0-9_]+)`?\s*\n", content)[1:]
    species_data = []

    for i in range(0, len(species_blocks), 2):
        species_name = species_blocks[i].strip()
        block = species_blocks[i + 1]

        # Extract description before "### Variables"
        desc_match = re.search(r"^\s*(.*?)\s*(?=### Variables|### Actions|$)", block, re.S)
        description = desc_match.group(1).strip() if desc_match else ""

        # Extract variables
        variables = []
        var_block = re.search(r"### Variables(.*?)(### Actions|----|$)", block, re.S)
        if var_block:
            for match in re.finditer(r"\*\*`(.+?)`\*\*\s*\((.+?)\):\s*(.*)", var_block.group(1)):
                var_name, var_type, var_desc = match.groups()
                variables.append({
                    "name": var_name.strip(),
                    "type": normalize_type_name(var_type.strip()),
                    "description": var_desc.strip()
                })

        # Extract actions
        actions = []
        action_block = re.search(r"### Actions(.*?)(----|$)", block, re.S)
        if action_block:
            action_parts = re.split(r"#### \*\*`(.+?)`\*\*", action_block.group(1))[1:]
            for j in range(0, len(action_parts), 2):
                action_name = action_parts[j].strip()
                details = action_parts[j + 1]

                desc_match = re.search(r"\*\*(.*?)\*\*", details, re.S)
                return_type_match = re.search(r"\*\*Returned type:\*\* `(.+?)`", details)
                example_match = re.search(r"##### Examples:\s*```(.*?)```", details, re.S)

                action_description = desc_match.group(1).strip() if desc_match else ""
                return_type = return_type_match.group(1).strip() if return_type_match else "unknown"
                example = example_match.group(1).strip() if example_match else None

                facets = []
                facets_block = re.search(r"##### Additional facets:(.*?)(?=\n####|\Z)", details, re.S)
                if facets_block:
                    for facet_match in re.finditer(r"\*\*`(.+?)`\*\* \((.+?)\):\s*(.*?)\s{2,}", facets_block.group(1)):
                        fname, ftype, fdesc = facet_match.groups()
                        facets.append({
                            "name": fname.strip(),
                            "type": ftype.strip(),
                            "description": fdesc.strip()
                        })

                actions.append({
                    "name": action_name,
                    "description": action_description,
                    "return_type": return_type,
                    "facets": facets,
                    "example": example
                })

        species_data.append({
            "name": species_name,
            "description": description,
            "variables": variables,
            "actions": actions
        })

    return species_data


def load_to_neo4j(species_data, name_target):
    with driver.session() as session:
        for species in species_data:
            session.run(f"""
                MERGE (s:{name_target} {{name: $name}})
                SET s.description = $description
            """, name=species["name"], description=species["description"])

            for var in species["variables"]:
                session.run(f"""
                    MATCH (s:{name_target} {{name: $species}})
                    CREATE (v:Variable {{
                        name: $name,
                        type: $type,
                        description: $desc
                    }})
                    MERGE (s)-[:HAS_VARIABLE]->(v)
                """, species=species["name"], name=var["name"], type=var["type"], desc=var["description"])

            for act in species["actions"]:
                session.run(f"""
                    MATCH (s:{name_target} {{name: $species}})
                    CREATE (a:Action {{
                        name: $name,
                        return_type: $return_type,
                        description: $desc,
                        example: $example
                    }})
                    MERGE (s)-[:HAS_ACTION]->(a)
                """, species=species["name"],
                    name=act["name"],
                    return_type=act["return_type"],
                    desc=act["description"],
                    example=act.get("example"))

                for facet in act.get("facets", []):
                    session.run("""
                        MATCH (a:Action {name: $action_name})
                        CREATE (f:Facet {
                            name: $name,
                            type: $type,
                            description: $desc
                        })
                        MERGE (a)-[:HAS_FACET]->(f)
                    """, action_name=act["name"], name=facet["name"], type=facet["type"], desc=facet["description"])

        session.run("""
                MATCH (v:Variable)
                WHERE v.type IS NOT NULL
                MATCH (t:Type {name: v.type})
                MERGE (v)-[:IS_OF_TYPE]->(t)
            """)
        session.run("""
                MATCH (a:Action)
                WHERE a.return_type IS NOT NULL
                MATCH (t:Type {name: a.return_type})
                MERGE (a)-[:RETURNS_TYPE]->(t)
            """)


if __name__ == "__main__":
    l = [["built_in_specie", "resources/BuiltInSpecies.md"],["built_in_skill", "resources/BuiltInSkills.md"],["built_in_architecture", "resources/BuiltInArchitectures.md"]]
    for name_target, file_path in l:
        species_data = parse_species_file(file_path)
        load_to_neo4j(species_data, name_target)

    driver.close()


# Filettypes

In [5]:
import re
from neo4j import GraphDatabase

# Neo4j connection details
uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"

driver = GraphDatabase.driver(uri, auth=(username, password))

import re

def parse_filetypes_file(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()

    blocks = re.split(r"\n##\s+(.*?)\n", content)[1:]
    filetypes = []

    for i in range(0, len(blocks), 2):
        filetype_name = blocks[i].strip()
        block = blocks[i + 1]

        # Extensions
        extensions = re.findall(r'"(.*?)"', block)

        # Content
        content_match = re.search(r"### Content\n(.*?)(?=###|\Z)", block, re.S)
        content_description = content_match.group(1).strip() if content_match else ""

        # Operators
        operators = []
        op_section_match = re.search(r"### Operators\n(.*?)(?=###|\Z)", block, re.S)
        if op_section_match:
            lines = op_section_match.group(1).splitlines()
            for line in lines:
                line = line.strip()
                if line.startswith("*"):
                    # Match: * **name(params)**: description...
                    # Allow stray quotes or closing punctuation after **)**
                    match = re.match(
                        r"^\*\s*\*\*([a-zA-Z0-9_\\]+)\((.*?)\)\*\*:\s*(.+)", line
                    )
                    if not match:
                        #print(f"⚠️ No match in signature: {line}")
                        continue

                    name = match.group(1).replace("\\_", "_")  #  This unescapes \_ to _
                    raw_params = match.group(2)
                    description = match.group(3).strip()
                    parameters = [p.strip() for p in raw_params.split(",") if p.strip()]
                    operators.append({
                        "name": name,
                        "parameters": parameters,
                        "description": description.strip()
                    })
                    #print(f"✅ Parsed: {name}({', '.join(parameters)})")

        filetypes.append({
            "name": filetype_name,
            "extensions": extensions,
            "content": content_description,
            "operators": operators
        })

    return filetypes




def load_filetypes_to_neo4j(filetypes):
    with driver.session() as session:
        for ft in filetypes:
            # Create FileType node
            session.run("""
                MERGE (f:FileType {name: $name})
                SET f.description = $content,
                    f.extensions = $extensions
            """, name=ft["name"], content=ft["content"], extensions=ft["extensions"])

            # Create associated Operator nodes
            for op in ft["operators"]:
                session.run("""
                    CREATE (o:Operator {
                        name: $name,
                        parameters: $params,
                        description: $desc
                    })
                    WITH o
                    MATCH (f:FileType {name: $ft_name})
                    MERGE (f)-[:HAS_OPERATOR]->(o)
                """,
                    name=op["name"],
                    params=op["parameters"],
                    desc=op["description"],
                    ft_name=ft["name"]
                )



if __name__ == "__main__":
    # ... previous loading for species/architectures ...

    # Load file types
    filetypes = parse_filetypes_file("resources/FileTypes.md")
    load_filetypes_to_neo4j(filetypes)

    driver.close()


# Statements
remplacé boolean par bool dans le fichier statements.md pour que le type soit reconnu
certaines facettes ont plusieurs types possibles, ici on considère que ce sont des facettes différentes. Faudra check si c'est une bonne chose

In [6]:
import re
from neo4j import GraphDatabase

# Neo4j connection details
uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"

driver = GraphDatabase.driver(uri, auth=(username, password))

def clear_db(tx):
    tx.run("MATCH (n) DETACH DELETE n")

def create_statement(tx, name, definition, usages, st_type): # ICI ON MET EXAMPLE AU LIEU D'USAGE, POSSIBLEMENT A MODIFIER PLUS TARD
    tx.run("""
        MERGE (s:Statement {name: $name})
        SET s.description = $definition,
            s.example = $usages,
            s.type = $st_type
    """, name=name, definition=definition, usages=usages, st_type=st_type)

def create_facet(tx, statement_name, facet_name, facet_type, description, optional, omissible):
    tx.run("""
        MATCH (s:Statement {name: $statement_name})
        CREATE (f:Facet {
            name: $facet_name,
            type: $facet_type,
            description: $description,
            omissible: $omissible,
            optional: $optional
        })
        MERGE (s)-[:HAS_FACET]->(f)
    """, statement_name=statement_name, facet_name=facet_name, facet_type=facet_type, description=description, omissible=omissible, optional=optional)

def create_embedment(tx, from_statement, to_statement):
    tx.run("""
        MATCH (a:Statement {name: $from_statement})
        MATCH (b:Statement {name: $to_statement})
        MERGE (a)-[:EMBEDS]->(b)
    """, from_statement=from_statement, to_statement=to_statement)

def create_embedding_context(tx, statement_name, context_name):
    tx.run("""
        MATCH (a:Statement {name: $statement_name})
        MATCH (b:Statement {name: $context_name})
        MERGE (a)-[:CAN_BE_EMBEDDED_INTO]->(b)
    """, statement_name=statement_name, context_name=context_name)

def link_facets_to_types(tx):
    tx.run("""
        MATCH (f:Facet)
        WHERE f.type IS NOT NULL
        MATCH (t:Type {name: f.type})
        MERGE (f)-[:OF_TYPE]->(t)
    """)

def parse_statements(md_text):
    entries = re.split(r'^###\s+', md_text, flags=re.MULTILINE)[1:]
    for entry in entries:
        lines = entry.strip().splitlines()
        name = lines[0].strip()

        # Parse definition
        definition_match = re.search(r"#### Definition\n\n(.+?)(\n\n|####)", entry, re.DOTALL)
        definition = definition_match.group(1).strip() if definition_match else ""

        # Parse usages
        usage_match = re.search(r"#### Usages\s*\n+(.*?)(?=\n##|\Z)", entry, re.DOTALL)
        usages = usage_match.group(1).strip() if usage_match else ""

        # Parse type
        type_match = re.search(r"statement is of type: \*\*(.+?)\*\*", entry)
        st_type = type_match.group(1).strip() if type_match else ""

        # Parse facets
        facet_lines = re.findall(
            r"^\*\s*(\*\*)?`([^`]+)`(\*\*)?\s*\(([^)]+)\)(,?\s*\(omissible\))?\s*:\s*(.+)",
            entry,
            re.MULTILINE
        )

        facets = []
        for is_bold_open, name_facet, is_bold_close, datatype, omissible_flag, desc in facet_lines:
            is_bold = bool(is_bold_open and is_bold_close)
            optional = not is_bold
            omissible = bool(omissible_flag and "omissible" in omissible_flag.lower())

            # Check if it's a list of types
            match = re.match(r'any type in\s*\[(.*?)\]', datatype)
            if match:
                types = [t.strip() for t in match.group(1).split(",")]
                for t in types:
                    facets.append((
                        name_facet.strip(),
                        t,
                        desc.strip(),
                        optional,
                        omissible
                    ))
            else:
                facets.append((
                    name_facet.strip(),
                    datatype.strip(),
                    desc.strip(),
                    optional,
                    omissible
                ))

        # Parse embedded statements
        embeds_match = re.search(r"embeds statements: (.+)", entry)
        embeds = re.findall(r"\[([^\]]+)\]", embeds_match.group(1)) if embeds_match else []

        # Parse embedding contexts
        embedded_into_match = re.search(r"can be embedded into: (.+)", entry)
        embedded_into = [s.strip() for s in re.split(r",\s*", embedded_into_match.group(1))] if embedded_into_match else []

        see_also_match = re.search(r"\* See also:\s*(.*?)(\n|$)", entry)
        see_also = []
        if see_also_match:
            see_also_links = see_also_match.group(1)
            see_also = re.findall(r"\[([^\]]+)\]", see_also_links)

        yield {
            "name": name,
            "definition": definition,
            "usages": usages,
            "type": st_type,
            "facets": facets,
            "embeds": embeds,
            "can_be_embedded_into": embedded_into,
            "see_also": see_also
        }

def link_related_statements(tx, from_stmt, to_stmt):
    tx.run("""
        MATCH (a:Statement {name: $origin})
        MATCH (b:Statement {name: $to})
        MERGE (a)-[:RELATED_TO]->(b)
    """, origin=from_stmt, to=to_stmt)


# MAIN EXECUTION
with driver.session() as session:
    # session.write_transaction(clear_db)

    with open("resources/Statements.md", encoding="utf-8") as f:
        content = f.read()
        # Use only content after ##General syntax
        content = content.split("##General syntax")[-1]

    for stmt in parse_statements(content):
        with driver.session() as session:
            session.write_transaction(create_statement, stmt["name"], stmt["definition"], stmt["usages"], stmt["type"])

            for facet_name, facet_type, desc, optional,omissible in stmt["facets"]:
                session.write_transaction(create_facet, stmt["name"], facet_name, facet_type, desc, optional, omissible)

            for embedded in stmt["embeds"]:
                session.write_transaction(create_embedment, stmt["name"], embedded)

            for context in stmt["can_be_embedded_into"]:
                session.write_transaction(create_embedding_context, stmt["name"], context)

    for stmt in parse_statements(content):
        with driver.session() as session:
            for related in stmt.get("see_also", []):
                session.write_transaction(link_related_statements, stmt["name"], related)

with driver.session() as session:
    session.write_transaction(link_facets_to_types)


/tmp/ipykernel_23216/569192952.py:153: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_statement, stmt["name"], stmt["definition"], stmt["usages"], stmt["type"])
/tmp/ipykernel_23216/569192952.py:156: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_facet, stmt["name"], facet_name, facet_type, desc, optional, omissible)
/tmp/ipykernel_23216/569192952.py:162: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_embedding_context, stmt["name"], context)
/tmp/ipykernel_23216/569192952.py:159: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_embedment, stmt["name"], embedded)
/tmp/ipykernel_23216/569192952.py:167: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(link_related_statements, stmt["name"], related)
/tmp/

# Units and constants

In [7]:
import re
from neo4j import GraphDatabase

# Neo4j connection
uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"
driver = GraphDatabase.driver(uri, auth=(username, password))

# Parser
def parse_units_constants(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()

    # Find all categories and their entries
    blocks = re.split(r"\n##+\s+([^\n]+)\n", content)[1:]
    entries = []

    for i in range(0, len(blocks), 2):
        category = blocks[i].strip()
        block = blocks[i + 1]

        # Match entries
        matches = re.findall(
            r"\*\*\s*`?#([a-zA-Z0-9_]+)`?.*?value=\s*([^,]+)?,?\s*Comment:\s*(.+)",
            block
        )

        for name, value, comment in matches:
            entries.append({
                "name": name.strip(),
                "category": category,
                "value": value.strip() if value else "No Default Value",
                "comment": comment.strip()
            })

    return entries

# Loader
def insert_units_constants(entries):
    with driver.session() as session:
        for entry in entries:
            session.run("""
                MERGE (n:ConstantOrUnit {
                    name: $name,
                    category: $category
                })
                SET n.value = $value,
                    n.description = $comment
            """, **entry)

# Run everything
entries = parse_units_constants("resources/UnitsAndConstants.md")
insert_units_constants(entries)
driver.close()


# Pseudo variables

In [8]:
import re
from neo4j import GraphDatabase

# Neo4j connection
uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"
driver = GraphDatabase.driver(uri, auth=(username, password))

def parse_pseudo_variables(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()

    # Match sections like ## self\n...content...
    blocks = re.split(r"\n##+\s+", content)[1:]
    entries = []

    for block in blocks:
        lines = block.strip().splitlines()
        if not lines:
            continue

        name = lines[0].strip()
        description_lines = []
        example_lines = []
        in_example = False

        for line in lines[1:]:
            if "```" in line:
                in_example = not in_example
                continue
            if in_example:
                example_lines.append(line)
            else:
                description_lines.append(line)

        description = " ".join([l.strip() for l in description_lines if l.strip()])
        example = "\n".join(example_lines).strip()

        entries.append({
            "name": name,
            "description": description,
            "example": example
        })

    return entries

def insert_pseudo_variables(entries):
    with driver.session() as session:
        for entry in entries:
            session.run("""
                MERGE (n:PseudoVariable {name: $name})
                SET n.description = $description,
                    n.example = $example
            """, **entry)

entries = parse_pseudo_variables("resources/PseudoVariables.md")
insert_pseudo_variables(entries)
driver.close()


# variables and attributes

In [9]:
import re
from neo4j import GraphDatabase

# Neo4j connection config
uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"
driver = GraphDatabase.driver(uri, auth=(username, password))

def extract_variable_info(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()

    entries = []

    # Extract code blocks and contexts
    code_blocks = re.findall(r"```\s*\n(.*?)```", content, re.S)
    full_code = "\n\n".join(code_blocks)

    # Extract variables mentioned in each section
    variable_sections = re.findall(r"### (.*?)\n(.*?)(?=\n### |\Z)", content, re.S)
    for context, desc_block in variable_sections:
        described_vars = re.findall(r"\* `([^`]+)`(?:,.*)?", desc_block)

        # Clean description block
        description = re.sub(r"`", "", desc_block).strip()

        for var in described_vars:
            entries.append({
                "name": var.strip(),
                "description": f"Used in context: {context}. {description}",
                "example": full_code  # optional: could narrow to relevant block
            })

    return entries

def insert_variable_nodes(entries):
    with driver.session() as session:
        for entry in entries:
            session.run("""
                MERGE (v:VariableOrAttribute {name: $name})
                SET v.description = $description,
                    v.example = $example
            """, **entry)

entries = extract_variable_info("resources/VariablesAndAttributes.md")
insert_variable_nodes(entries)
driver.close()


# Operators

In [10]:
import re
from neo4j import GraphDatabase

# Neo4j connection
uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"
driver = GraphDatabase.driver(uri, auth=(username, password))

def normalize_type(type_str):
    type_str = type_str.strip().strip("`")
    type_str = re.sub(r"<.*?>", "", type_str)
    type_str = re.sub(r"[>`]+$", "", type_str)
    return type_str.strip()

def extract_operations(block):
    operations = []
    uses_match = re.search(r"#### Possible uses:\s*(.*?)(?=\n####|\Z)", block, re.S)
    if not uses_match:
        return operations

    lines = uses_match.group(1).strip().splitlines()
    for line in lines:
        original_line = line.strip()
        if not original_line:
            continue

        match = re.match(r"[*•]\s*(.*?)--->\s*`?([^`]+)`?", original_line)
        if not match:
            print(f"⚠️ Could not parse line: {original_line}")
            continue

        signature = match.group(1).strip()
        output_type = normalize_type(match.group(2))
        backticked = re.findall(r"`([^`]+)`", original_line)

        if len(backticked) < 2:
            print(f"⚠️ Not enough backticked tokens in line: {original_line}")
            continue

        name = original_line
        has_parens = "(" in signature and ")" in signature
        operator, input_types, \
        first_input_type, second_input_type, third_input_type, fourth_input_type, fifth_input_type, \
        sixth_input_type, seventh_input_type, eighth_input_type, ninth_input_type, tenth_input_type, \
        eleventh_input_type, twelfth_input_type, thirteenth_input_type = None, [], \
        None, None, None, None, None, \
        None, None, None, None, None, \
        None, None, None

        all_inputs = {
            "first_input_type": first_input_type,
            "second_input_type": second_input_type,
            "third_input_type": third_input_type,
            "fourth_input_type": fourth_input_type,
            "fifth_input_type": fifth_input_type,
            "sixth_input_type": sixth_input_type,
            "seventh_input_type": seventh_input_type,
            "eighth_input_type": eighth_input_type,
            "ninth_input_type": ninth_input_type,
            "tenth_input_type": tenth_input_type,
            "eleventh_input_type": eleventh_input_type,
            "twelfth_input_type": twelfth_input_type,
            "thirteenth_input_type": thirteenth_input_type
        }


        if has_parens and len(backticked) >=4 and len(backticked)<=15:
            operator = backticked[0]
            input_keys = list(all_inputs.keys())
            for i in range(len(backticked)-2):
                #all_inputs[input_keys[i]] = normalize_type(backticked[i + 1])
                val = normalize_type(backticked[i + 1])
                all_inputs[input_keys[i]] = val
                locals()[input_keys[i]] = val 
            input_types = [v for v in all_inputs.values() if v is not None]
            output_type = normalize_type(backticked[-1])

        elif len(backticked) == 3:
            operator = backticked[0]
            all_inputs['first_input_type'] = normalize_type(backticked[1])
            input_types = [first_input_type]
            output_type = normalize_type(backticked[2])

        elif not has_parens and len(backticked) == 4:
            all_inputs['first_input_type'] = normalize_type(backticked[0])
            operator = backticked[1]
            all_inputs['second_input_type'] = normalize_type(backticked[2])
            input_types = [first_input_type, second_input_type]
            output_type = normalize_type(backticked[-1])

        
        else:
            print(f"⚠️ Ambiguous or unsupported format in line: {original_line}")
            continue

        operations.append({
            "name": name,
            "operator": operator,
            "inputs": input_types,
            "output": output_type,
            "first_input_type": all_inputs["first_input_type"],
            "second_input_type": all_inputs["second_input_type"],
            "third_input_type": all_inputs["third_input_type"],
            "fourth_input_type": all_inputs["fourth_input_type"],
            "fifth_input_type": all_inputs["fifth_input_type"],
            "sixth_input_type": all_inputs["sixth_input_type"],
            "seventh_input_type": all_inputs["seventh_input_type"],
            "eighth_input_type": all_inputs["eighth_input_type"],
            "ninth_input_type": all_inputs["ninth_input_type"],
            "tenth_input_type": all_inputs["tenth_input_type"],
            "eleventh_input_type": all_inputs["eleventh_input_type"],
            "twelfth_input_type": all_inputs["twelfth_input_type"],
            "thirteenth_input_type": all_inputs["thirteenth_input_type"]
        })


    return operations

def get_existing_types(driver):
    with driver.session() as session:
        result = session.run("MATCH (t:Type) RETURN t.name AS name")
        return set(record["name"] for record in result)

def parse_operators(path, existing_types):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()

    blocks = re.split(r"\n###\s+`(.+?)`\n", content)[1:]
    entries = []
    missing_types = set()
    alias_links = []

    for i in range(0, len(blocks), 2):
        op_name = blocks[i].strip()
        block = blocks[i + 1]

        # Detect alias like: "Same signification as [-](OperatorsAA#-)"
        alias_match = re.search(r"Same signification as\s+\[([^\]]+)\]", block)
        if alias_match:
            target_name = alias_match.group(1).strip()
            alias_links.append((op_name, target_name))
            continue  # Skip full parsing for alias, since it's just a redirect

        operations = extract_operations(block)

        for op in operations:
            for t in sorted(t for t in missing_types if t is not None):
                if t not in existing_types:
                    missing_types.add(t)

        result = re.search(r"#### Result:\s*(.*?)(?=\n####|\Z)", block, re.S)
        comment = re.search(r"#### Comment:\s*(.*?)(?=\n####|\Z)", block, re.S)
        special_cases = re.search(r"#### Special cases:\s*(.*?)(?=\n####|\Z)", block, re.S)
        example = re.search(r"#### Examples:\s*(.*?)(?=\n####|\Z)", block, re.S)
        see_also_match = re.search(r"\*\*See also:\*\*\s*(.*?)(?=\n|----|\Z)", block, re.S)
        see_also_links = []
        if see_also_match:
            see_also_text = see_also_match.group(1)
            see_also_links = re.findall(r"\[([^\]]+)\]\([^)]+\)", see_also_text)

        entries.append({
            "name": op_name,
            "result": result.group(1).strip() if result else "",
            "comment": comment.group(1).strip() if comment else "",
            "special_cases": re.sub(r"```(.*?)```", r"\1", special_cases.group(1).strip(), flags=re.S) if special_cases else "",
            "example": re.sub(r"```(.*?)```", r"\1", example.group(1).strip(), flags=re.S) if example else "",
            "operations": operations,
            "see_also": see_also_links
        })

    if missing_types:
        print("⚠️ Unknown types used in operations (not found in DB):")
        for t in sorted(missing_types):
            print(op_name + f" - {t}")

    return entries, alias_links

def insert_aliases_into_neo4j(alias_links, driver):
    with driver.session() as session:
        for alias_name, target_name in alias_links:
            result = session.run("""
                MATCH (target:Operator {name: $target_name})
                WITH target
                MERGE (alias:Operator {name: $alias_name})
                MERGE (alias)-[:ALIAS_OF]->(target)
                RETURN alias.name AS alias, target.name AS target
            """, alias_name=alias_name, target_name=target_name)

            if not result.peek():
                print(f"⚠️ Could not create alias: '{alias_name}' → '{target_name}' (target not found)")
            else:
                print(f"✅ Created alias: '{alias_name}' → '{target_name}'")


def insert_into_neo4j(entries):
    existing_types = get_existing_types(driver)
    with driver.session() as session:
        for entry in entries:
            session.run("""
                MERGE (o:Operator {name: $name})
                SET o.description = $result,
                    o.comment = $comment,
                    o.special_cases = $special_cases,
                    o.example=$example
            """, name=entry["name"], result=entry["result"],
                 comment=entry["comment"], special_cases=entry["special_cases"], example=entry["example"])

            for op_data in entry["operations"]:
                session.run("""
                    MATCH (o:Operator {name: $op_name})
                    MERGE (p:PossibleOperation {name: $line})
                    SET p.output = $output,
                        p.inputs = $inputs,
                        p.description = $line
                    MERGE (o)-[:HAS_OPERATION]->(p)

                    WITH p
                    OPTIONAL MATCH (t1:Type {name: $first_input_type})
                    FOREACH (_ IN CASE WHEN t1 IS NOT NULL THEN [1] ELSE [] END |
                        MERGE (p)-[:TAKES_FIRST_INPUT_TYPE]->(t1)
                    )

                    WITH p
                    OPTIONAL MATCH (t2:Type {name: $second_input_type})
                    FOREACH (_ IN CASE WHEN t2 IS NOT NULL THEN [1] ELSE [] END |
                        MERGE (p)-[:TAKES_SECOND_INPUT_TYPE]->(t2)
                    )

                    WITH p
                    OPTIONAL MATCH (t3:Type {name: $third_input_type})
                    FOREACH (_ IN CASE WHEN t3 IS NOT NULL THEN [1] ELSE [] END |
                        MERGE (p)-[:TAKES_THIRD_INPUT_TYPE]->(t3)
                    )

                    WITH p
                    OPTIONAL MATCH (t4:Type {name: $fourth_input_type})
                    FOREACH (_ IN CASE WHEN t4 IS NOT NULL THEN [1] ELSE [] END |
                        MERGE (p)-[:TAKES_FOURTH_INPUT_TYPE]->(t4)
                    )

                    WITH p
                    OPTIONAL MATCH (t5:Type {name: $fifth_input_type})
                    FOREACH (_ IN CASE WHEN t5 IS NOT NULL THEN [1] ELSE [] END |
                        MERGE (p)-[:TAKES_FIFTH_INPUT_TYPE]->(t5)
                    )

                    WITH p
                    OPTIONAL MATCH (t6:Type {name: $sixth_input_type})
                    FOREACH (_ IN CASE WHEN t6 IS NOT NULL THEN [1] ELSE [] END |
                        MERGE (p)-[:TAKES_SIXTH_INPUT_TYPE]->(t6)
                    )

                    WITH p
                    OPTIONAL MATCH (t7:Type {name: $seventh_input_type})
                    FOREACH (_ IN CASE WHEN t7 IS NOT NULL THEN [1] ELSE [] END |
                        MERGE (p)-[:TAKES_SEVENTH_INPUT_TYPE]->(t7)
                    )

                    WITH p
                    OPTIONAL MATCH (t8:Type {name: $eighth_input_type})
                    FOREACH (_ IN CASE WHEN t8 IS NOT NULL THEN [1] ELSE [] END |
                        MERGE (p)-[:TAKES_EIGHTH_INPUT_TYPE]->(t8)
                    )

                    WITH p
                    OPTIONAL MATCH (t9:Type {name: $ninth_input_type})
                    FOREACH (_ IN CASE WHEN t9 IS NOT NULL THEN [1] ELSE [] END |
                        MERGE (p)-[:TAKES_NINTH_INPUT_TYPE]->(t9)
                    )

                    WITH p
                    OPTIONAL MATCH (t10:Type {name: $tenth_input_type})
                    FOREACH (_ IN CASE WHEN t10 IS NOT NULL THEN [1] ELSE [] END |
                        MERGE (p)-[:TAKES_TENTH_INPUT_TYPE]->(t10)
                    )

                    WITH p
                    OPTIONAL MATCH (t11:Type {name: $eleventh_input_type})
                    FOREACH (_ IN CASE WHEN t11 IS NOT NULL THEN [1] ELSE [] END |
                        MERGE (p)-[:TAKES_ELEVENTH_INPUT_TYPE]->(t11)
                    )

                    WITH p
                    OPTIONAL MATCH (t12:Type {name: $twelfth_input_type})
                    FOREACH (_ IN CASE WHEN t12 IS NOT NULL THEN [1] ELSE [] END |
                        MERGE (p)-[:TAKES_TWELFTH_INPUT_TYPE]->(t12)
                    )

                    WITH p
                    OPTIONAL MATCH (t13:Type {name: $thirteenth_input_type})
                    FOREACH (_ IN CASE WHEN t13 IS NOT NULL THEN [1] ELSE [] END |
                        MERGE (p)-[:TAKES_THIRTEENTH_INPUT_TYPE]->(t13)
                    )


                    WITH p, $output AS output_type
                    MATCH (t_out:Type {name: output_type})
                    MERGE (p)-[:RETURNS_TYPE]->(t_out)
                """, op_name=entry["name"],
                     line=op_data["name"],
                     inputs=[i for i in op_data["inputs"] if i is not None],
                     output=op_data["output"],
                     first_input_type=op_data["first_input_type"],
                     second_input_type=op_data["second_input_type"],
                     third_input_type=op_data["third_input_type"],
                     fourth_input_type=op_data["fourth_input_type"],
                     fifth_input_type=op_data["fifth_input_type"],
                     sixth_input_type=op_data["sixth_input_type"],
                     seventh_input_type=op_data["seventh_input_type"],
                     eighth_input_type=op_data["eighth_input_type"],
                     ninth_input_type=op_data["ninth_input_type"],
                     tenth_input_type=op_data["tenth_input_type"],
                     eleventh_input_type=op_data["eleventh_input_type"],
                     twelfth_input_type=op_data["twelfth_input_type"],
                     thirteenth_input_type=op_data["thirteenth_input_type"])

                # Debug output
                for rel_type, key in [("TAKES_FIRST_INPUT_TYPE", "first_input_type"), ("TAKES_SECOND_INPUT_TYPE", "second_input_type")]:
                    input_type = op_data.get(key)
                    if input_type and input_type in existing_types:
                        result = session.run(f"""
                            MATCH (p:PossibleOperation {{name: $line}})
                            MATCH (t:Type {{name: $type}})
                            WHERE NOT (p)-[:{rel_type}]->(t)
                            RETURN p.name AS pname, t.name AS tname
                        """, line=op_data["name"], type=input_type).data()
                        if result:
                            print(f"⚠️ {rel_type} NOT CREATED despite existing type: {result[0]['pname']} -> {result[0]['tname']}")
        for entry in entries:
            for related_name in entry.get("see_also", []):
                session.run("""
                    MATCH (a:Operator {name: $origin})
                    MATCH (b:Operator {name: $to})
                    MERGE (a)-[:RELATED_TO]->(b)
                """, origin=entry["name"], to=related_name)


# Run the script
entries, alias_links = parse_operators("resources/OperatorsSplitted.md", get_existing_types(driver))
insert_into_neo4j(entries)
insert_aliases_into_neo4j(alias_links, driver)


with driver.session() as session:
    session.run("""
MATCH (n:Operator)
WHERE n.description is null
SET n.description=n.name
    """)


driver.close()


⚠️ Could not parse line: ----
⚠️ Could not parse line: ----
⚠️ Could not parse line: ----
⚠️ Could not parse line: ----
⚠️ Could not parse line: ----
⚠️ Could not parse line: ----
⚠️ Could not parse line: ----
⚠️ Could not parse line: ----
✅ Created alias: '@' → 'at'
✅ Created alias: 'add_days' → 'plus_days'
✅ Created alias: 'add_hours' → 'plus_hours'
✅ Created alias: 'add_minutes' → 'plus_minutes'
✅ Created alias: 'add_months' → 'plus_months'
✅ Created alias: 'add_ms' → 'plus_ms'
✅ Created alias: 'add_seconds' → '+'
✅ Created alias: 'add_weeks' → 'plus_weeks'
✅ Created alias: 'add_years' → 'plus_years'
✅ Created alias: 'agent_intersecting' → 'agents_overlapping'
✅ Created alias: 'all_verify' → 'all_match'
✅ Created alias: 'any' → 'one_of'
✅ Created alias: 'any_point_in' → 'any_location_in'
✅ Created alias: 'buffer' → '+'
✅ Created alias: 'contains_node' → 'contains_key'
✅ Created alias: 'contains_value' → 'contains'
✅ Created alias: 'cropped_to' → 'clipped_with'
✅ Created alias: 'deco

## Global specie

In [11]:
import re
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"
driver = GraphDatabase.driver(uri, auth=(username, password))

def parse_global_species_markdown(text):
    attr_section = re.search(r'## Built-in attributes(.*?)## Built-in Actions', text, flags=re.DOTALL)
    action_section = re.search(r'## Built-in Actions(.*?)### Other actions', text, flags=re.DOTALL)

    attributes = extract_sections(attr_section.group(1)) if attr_section else []
    actions = extract_sections(action_section.group(1)) if action_section else []

    #print(actions)

    return attributes, actions


def insert_global_species(session, attributes, actions):
    session.run("""
        MERGE (g:Global {name: "global"})
    """)

    for attr in attributes:
        session.run("""
            MERGE (a:Attribute {name: $name, type: "BuiltInAttribute"})
            SET a.example = $example, a.content = $content
            MERGE (g:Global {name: "global"})
            MERGE (g)-[:HAS_ATTRIBUTE]->(a)
        """, name=attr["name"], example=attr.get("example"), content=attr.get("content"))

    for act in actions:
        session.run("""
            MERGE (a:Action {name: $name, type: "BuiltInAction"})
            SET a.example = $example, a.content = $content
            MERGE (g:Global {name: "global"})
            MERGE (g)-[:HAS_ACTION]->(a)
        """, name=act["name"], example=act.get("example"), content=act.get("content"))


def extract_sections(section_text):
    pattern = r'### `(\w+)`\s*\n(.*?)(?=### `\w+`|\Z)'
    entries = re.findall(pattern, section_text, flags=re.DOTALL)

    result = []

    for name, block in entries:
        # Find all code blocks and their positions
        code_blocks = list(re.finditer(r'```(?:\w*\n)?(.*?)```', block, flags=re.DOTALL))

        # Extract all code block contents
        examples = [m.group(1).strip() for m in code_blocks]
        example_combined = "\n\n".join(examples) if examples else None

        # Extract content lines starting with '*'
        content_lines = [line.strip() for line in block.strip().splitlines() if line.strip().startswith("*")]
        content = "\n".join(content_lines) if content_lines else None

        # Remove code blocks from the block to isolate remaining explanation text
        block_cleaned = block
        for m in code_blocks:
            block_cleaned = block_cleaned.replace(m.group(0), '')

        # Remove lines already captured in `content`
        block_lines = [line.strip() for line in block_cleaned.strip().splitlines()]
        explanation_lines = [
            line for line in block_lines
            if line and not line.startswith("*")
        ]
        explanation = "\n"+"\n".join(explanation_lines).strip() or ''

        
        result.append({
            "name": name,
            "example": example_combined,
            "content": content+explanation
        })

    return result




with open("resources/GlobalSpecies.md", "r", encoding="utf-8") as f:
    content = f.read()

attributes, actions = parse_global_species_markdown(content)

with driver.session() as session:
    insert_global_species(session, attributes, actions)



## Regular Species

In [12]:
import re
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"
driver = GraphDatabase.driver(uri, auth=(username, password))

def extract_list_section(section_text):
    lines = [line.strip() for line in section_text.strip().splitlines()]
    return [line for line in lines if line.startswith("*")]

def parse_regular_species_markdown(text):
    agent_attr_section = re.search(r'## Built-in attributes(.*?)## Species built-in Attributes', text, flags=re.DOTALL)
    species_attr_section = re.search(r'## Species built-in Attributes(.*?)## Built-in actions', text, flags=re.DOTALL)


    agent_attrs = extract_list_section(agent_attr_section.group(1)) if agent_attr_section else []
    species_attrs = extract_list_section(species_attr_section.group(1)) if species_attr_section else []

    # Get category examples from ``` blocks
    agent_example = "\n\n".join(re.findall(r'```(?:\w*\n)?(.*?)```', agent_attr_section.group(1), flags=re.DOTALL)) if agent_attr_section else None
    species_example = "\n\n".join(re.findall(r'```(?:\w*\n)?(.*?)```', species_attr_section.group(1), flags=re.DOTALL)) if species_attr_section else None

    # Format entries
    def format_entry(lines, label, example):
        result = []
        for line in lines:
            name_match = re.match(r'\*+\s+\*\*(\w+)`?', line)
            type_match = re.search(r'\(type:\s*([\w\d_]+)', line)
            name = name_match.group(1) if name_match else "unknown"
            type_value = type_match.group(1) if type_match else "unknown"
            result.append({
                "name": name,
                "type": type_value,
                "content": line.strip('* ').strip(),
                "example": example
            })
        return result


    agent_attr_nodes = format_entry(agent_attrs, "BuiltInAgentAttribute", agent_example)
    species_attr_nodes = format_entry(species_attrs, "BuiltInSpeciesAttribute", species_example)

    return agent_attr_nodes, species_attr_nodes

def insert_regular_species(session, agent_attrs, species_attrs):
    session.run("""MERGE (s:Species {name: "species"})""")

    def insert_nodes(label, entries, rel_type):
        for item in entries:
            session.run(f"""
                MERGE (n:{label} {{name: $name}})
                SET n.type = $type, n.content = $content, n.example = $example
                WITH n
                MATCH (s:Species {{name: "species"}})
                MERGE (s)-[:{rel_type}]->(n)
            """, **item)

    insert_nodes("BuiltInAgentAttribute", agent_attrs, "HAS_ATTRIBUTE")
    insert_nodes("BuiltInSpeciesAttribute", species_attrs, "HAS_ATTRIBUTE")


def link_attributes_to_types(session):
    session.run("""
        // Match all nodes with a 'type' property, across both attribute labels
        MATCH (a)
        WHERE a.type IS NOT NULL AND a:BuiltInAgentAttribute OR a:BuiltInSpeciesAttribute
        MERGE (t:Type {name: a.type})
        MERGE (a)-[:OF_TYPE]->(t)
    """)

with open("resources/RegularSpecies.md", "r", encoding="utf-8") as f:
    text = f.read()

agent_attrs, species_attrs = parse_regular_species_markdown(text)

with driver.session() as session:
    insert_regular_species(session, agent_attrs, species_attrs)
    link_attributes_to_types(session)


### Duplication des relations de regular species à global specie

In [13]:
def replicate_species_attributes_to_global(session):
    session.run("""
        MATCH (:Species)-[:HAS_ATTRIBUTE]->(a)
        MERGE (g:Global {name: "global"})
        MERGE (g)-[:HAS_ATTRIBUTE]->(a)
    """)

with driver.session() as session:
    insert_regular_species(session, agent_attrs, species_attrs)
    link_attributes_to_types(session)
    replicate_species_attributes_to_global(session)

Creation de all_content

In [14]:
import warnings
import numpy as np
from neo4j import GraphDatabase
from langchain_ollama import OllamaEmbeddings
from langchain_core._api.deprecation import LangChainDeprecationWarning


uri = "bolt://localhost:7687"
user = "neo4j"
password = "password"

driver = GraphDatabase.driver(uri, auth=(user, password))

def get_nodes_with_properties(tx):
    result = tx.run("MATCH (n) RETURN id(n) AS id, properties(n) AS props")
    return [(record["id"], record["props"]) for record in result]

def format_properties(props):
    formatted = []
    for k, v in props.items():
        if k == "all_content":
            continue  # skip self-reference
        if isinstance(v, list):
            val_str = " ".join(str(item) for item in v)
        else:
            val_str = str(v)
        formatted.append(f"{k}: {val_str}")
    return " ".join(formatted)

def update_all_content(tx, node_id, all_content):
    tx.run("MATCH (n) WHERE id(n) = $id SET n.all_content = $content SET n.weight=0",
           id=node_id, content=all_content)

def get_distinct_keys(tx):
    result = tx.run("""
        MATCH (n)
        UNWIND keys(n) AS key
        RETURN DISTINCT key
    """)
    return [record["key"] for record in result]

def build_key_value_concat(keys):
    parts = []
    for key in keys:
        if key == "all_content":
            continue  # avoid self-reference
        # Builds: "key: " + (isList ? join(...) : toString(...))
        part = (
            f'"{key}: " + '
            f'CASE '
            f'WHEN n.{key} IS NULL THEN "" '
            f'WHEN n.{key}[0] IS NOT NULL THEN apoc.text.join([x IN n.{key} | toString(x)], " ") '
            f'ELSE toString(n.{key}) '
            f'END'
        )
        parts.append(part)
    return " + ' ' + ".join(parts)


def set_all_content(session, keys):
    content_expr = build_key_value_concat(keys)
    query = f"""
        MATCH (n)
        SET n.all_content = {content_expr}
    """
    session.run(query)

with driver.session() as session:
    nodes = session.execute_read(get_nodes_with_properties)
    for node_id, props in nodes:
        content = format_properties(props)
        session.execute_write(update_all_content, node_id, content)

driver.close()

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 1, column: 18, offset: 17} for query: 'MATCH (n) RETURN id(n) AS id, properties(n) AS props'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 1, column: 17, offset: 16} for query: 'MATCH (n) WHERE id(n) = $id SET n.all_content = $content SET n.weight=0'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and